#### Flask연동하기

In [1]:
# 분석용 서버 구축을 위한 flask 설치
!pip install flask

In [2]:
# 영상처리 분야에서 많이 쓰이는 라이브러리
!pip install opencv-python

In [3]:
from flask import Flask, request, Response, redirect
import io # 파이썬의 입출력을 도와주는 라이브러리
import numpy as np
import matplotlib.pyplot as plt
import cv2 # opencv
import pickle # 모델로딩용

In [4]:
# 서버생성
app = Flask(__name__)

# 사용자의 요청에따라 처리하는 라우터
@app.route('/', methods=['GET','POST'])
def index() :
    # GET방식 처리 코드
    if request.method == 'GET' :
        print(request.args)       
        return 'Hello {}'.format(request.args['name'])
    else : # POST방식 처리 코드
        return 'Hello'

@app.route('/bye/<name>', methods=['GET','POST'])
def bye(name) :
    return 'bye bye {}'.format(name)


app.run(host='192.168.21.104',port=5001) # 서버구동

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://192.168.21.104:5001/ (Press CTRL+C to quit)


#### 손글씨 예측 서버 구축

In [12]:
# 학습된 모델 객체 로딩
with open('./digit_model.pkl', 'rb') as f:
    model = pickle.load(f)
# 분석용 서버 객체 생성
app = Flask(__name__)

@app.route("/predict", methods=['POST'])
def predict():
    # 사진을 받아서 전처리
    if request.method == 'POST':
        print(request.files)
        img = request.files['img']
        input_stream = io.BytesIO() # byte 단위로 읽어들이는 통로
        img.save(input_stream) # 파일로부터 데이터를 읽기
        data = np.fromstring(input_stream.getvalue(),
                            dtype = np.uint8)
        # byte 단위의 numpy 데이터를 opencv를 이용해 이미지로 변환
        real_img = cv2.imdecode(data,cv2.IMREAD_GRAYSCALE)
        # BGR -> RGB
        #real_img_rbg = cv2.cvtColor(real_img, cv2.COLOR_BGR2RGB)
        #display(real_img_rbg)
        #plt.imshow(real_img_rbg)
        #plt.show()
        
        # 모델 예측
        resize_img = cv2.resize(real_img, dsize=(28,28),
                               interpolation=cv2.INTER_AREA)
        
        # 만약에 학습데이터가 스케일링이 되었으면 
        # 예측 데이터도 스케일링 작업을 해줘야 함.
        
        # 모델이 예측하기 위한 numpy 타입으로 변경
        test =np.array(resize_img).reshape(1,784)
        
        # 모델 예측
        pre = model.predict(test)
  
    return  redirect("http://localhost:8081/HandDiditService/result.jsp?pre="+str(pre[0]))  

# 서버 실행
app.run(host='192.168.21.104',port=5001)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://192.168.21.104:5001/ (Press CTRL+C to quit)
C:\Users\aischool\AppData\Local\Temp\ipykernel_66604\2595600901.py:15: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(input_stream.getvalue(),
C:\Users\aischool\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
192.168.21.104 - - [06/Oct/2022 10:35:42] "POST /predict HTTP/1.1" 302 -


ImmutableMultiDict([('img', <FileStorage: '7.png' ('image/png')>)])


C:\Users\aischool\AppData\Local\Temp\ipykernel_66604\2595600901.py:15: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(input_stream.getvalue(),
C:\Users\aischool\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
192.168.21.104 - - [06/Oct/2022 10:35:56] "POST /predict HTTP/1.1" 302 -


ImmutableMultiDict([('img', <FileStorage: '2.png' ('image/png')>)])
